In [ ]:
import json

# Example with a non-ASCII character in the data
data = {'name': 'John Doe', 'city': '北京'}

# Convert Python object to JSON string
json_string = json.dumps(data)

# Encode JSON string to bytes using UTF-8
json_bytes = json_string.encode('utf-8')

# Now you can safely use `json_bytes` for writing to file or transmitting over network
print(json_bytes)

In [1]:
import uuid

In [3]:
str(uuid.uuid4())

'18512de5-26b5-43b2-b9fc-0b4dc959a33d'

In [16]:
spark.stop()

In [47]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName('sudhirzen')
    .config('spark.jars.packages', "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")
    .getOrCreate()
)

24/07/06 13:29:54 WARN Utils: Your hostname, iPhone.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
24/07/06 13:29:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/crepantherx/.ivy2/cache
The jars for the packages stored in: /Users/crepantherx/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2815c472-7cfb-41bf-ae8b-8156ce9036a7;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in cen

In [48]:
df = spark.readStream \
.format('kafka') \
.option('kafka.bootstrap.servers', 'localhost:9092') \
.option('subscribe', 'users_created') \
.option('startingOffsets', 'earliest') \
.load()

In [49]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
schema = StructType([
    StructField("id", StringType(), False),
    StructField("first_name", StringType(), False),
    StructField("last_name", StringType(), False),
    StructField("gender", StringType(), False),
    StructField("address", StringType(), False),
    StructField("post_code", StringType(), False),
    StructField("email", StringType(), False),
    StructField("username", StringType(), False),
    StructField("registered_date", StringType(), False),
    StructField("phone", StringType(), False),
    StructField("picture", StringType(), False)
])

In [10]:
df.start().selectExpr("CAST(value AS STRING)").select(from_json(col('value'), schema).alias('data')).select("data.*").show()

AttributeError: 'DataFrame' object has no attribute 'start'

In [15]:
df.selectExpr("CAST(value AS STRING)") \
  .select(from_json(col('value'), schema).alias('data')) \
  .select("data.*").writeStream.format("console").start()

24/07/05 20:41:16 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0d/q901nqg95plg2cz08tmt75480000gn/T/temporary-a72ef63d-2b60-4c53-bc3f-ef919853dba4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/05 20:41:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [9]:
import requests
import json

In [20]:
res = requests.get("https://randomuser.me/api/").json()['results'][0]

In [22]:
res = res.json()['results'][0]

In [13]:
a = json.dumps(result.json()['results'][0])

In [19]:
a.encode('utf-8')

b'{"gender": "female", "name": {"title": "Mrs", "first": "Kristin", "last": "Perkins"}, "location": {"street": {"number": 4263, "name": "Railroad St"}, "city": "Nowra", "state": "Western Australia", "country": "Australia", "postcode": 4189, "coordinates": {"latitude": "-82.0315", "longitude": "31.5765"}, "timezone": {"offset": "-1:00", "description": "Azores, Cape Verde Islands"}}, "email": "kristin.perkins@example.com", "login": {"uuid": "8b766a9e-7780-4a6b-96ec-711efdb5d2a2", "username": "smallladybug174", "password": "tickle", "salt": "EFbOzUhy", "md5": "d9396ee1484baef3023ff74fe097db20", "sha1": "d8629ef8471ddb3adeee9d5ee998dd6b644c6a28", "sha256": "a5381a71f0e59579866d4c73235b0a05a90585d1246c8592260b21be89d780f6"}, "dob": {"date": "1950-06-27T18:57:02.596Z", "age": 74}, "registered": {"date": "2020-03-25T01:58:17.982Z", "age": 4}, "phone": "08-7381-6186", "cell": "0441-236-346", "id": {"name": "TFN", "value": "833297907"}, "picture": {"large": "https://randomuser.me/api/portraits/

In [24]:
import uuid

In [42]:
res = requests.get("https://randomuser.me/api/").json()['results'][0]

data = {}
location = res['location']
data['id'] = str(uuid.uuid4())
data['first_name'] = res['name']['first']
data['last_name'] = res['name']['last']
data['gender'] = res['gender']
data['address'] = f"{str(location['street']['number'])} {location['street']['name']}, " \
                  f"{location['city']}, {location['state']}, {location['country']}"
data['post_code'] = location['postcode']
data['email'] = res['email']
data['username'] = res['login']['username']
data['dob'] = res['dob']['date']
data['registered_date'] = res['registered']['date']
data['phone'] = res['phone']
data['picture'] = res['picture']['medium']


In [34]:
json.dumps(data).encode('utf-8')

b'{"id": "455a238f-fcb6-4220-af0e-4ad11c9adfc4", "first_name": "Mahika", "last_name": "Prabhakaran", "gender": "female", "address": "6372 Gandhi Maidan Marg, Jabalpur, Madhya Pradesh, India", "post_code": 50991, "email": "mahika.prabhakaran@example.com", "username": "silverfish226", "dob": "1946-09-04T11:37:03.976Z", "registered_date": "2013-04-17T09:35:17.577Z", "phone": "9123140326", "picture": "https://randomuser.me/api/portraits/med/women/59.jpg"}'

In [35]:
from kafka import KafkaProducer

In [43]:
k = json.dumps(data).encode('utf-8')

In [38]:
k

b'{"id": "455a238f-fcb6-4220-af0e-4ad11c9adfc4", "first_name": "Mahika", "last_name": "Prabhakaran", "gender": "female", "address": "6372 Gandhi Maidan Marg, Jabalpur, Madhya Pradesh, India", "post_code": 50991, "email": "mahika.prabhakaran@example.com", "username": "silverfish226", "dob": "1946-09-04T11:37:03.976Z", "registered_date": "2013-04-17T09:35:17.577Z", "phone": "9123140326", "picture": "https://randomuser.me/api/portraits/med/women/59.jpg"}'

In [40]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], max_block_ms=5000)

In [46]:
producer.send('amruta', k)

In [51]:
spark_df = spark.readStream \
            .format('kafka') \
            .option('kafka.bootstrap.servers', 'localhost:9092') \
            .option('subscribe', 'users_created') \
            .option('startingOffsets', 'earliest') \
            .load()

In [52]:
schema = StructType([
    StructField("id", StringType(), False),
    StructField("first_name", StringType(), False),
    StructField("last_name", StringType(), False),
    StructField("gender", StringType(), False),
    StructField("address", StringType(), False),
    StructField("post_code", StringType(), False),
    StructField("email", StringType(), False),
    StructField("username", StringType(), False),
    StructField("registered_date", StringType(), False),
    StructField("phone", StringType(), False),
    StructField("picture", StringType(), False)
])

sel = spark_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col('value'), schema).alias('data')).select("data.*")

In [53]:
sel.writeStream.format("console").start().awaitTermination()

24/07/06 13:44:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0d/q901nqg95plg2cz08tmt75480000gn/T/temporary-2e0ba285-ece0-4caf-9fb1-477363481bc4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/06 13:44:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/07/06 13:44:26 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/07/06 13:44:26 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/07/06 13:44:26 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/07/06 13:44:26 WARN AdminClientConfig: The configuration '

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+----------+----------+------+--------------------+---------+--------------------+------------------+--------------------+--------------+--------------------+
|     id|first_name| last_name|gender|             address|post_code|               email|          username|     registered_date|         phone|             picture|
+-------+----------+----------+------+--------------------+---------+--------------------+------------------+--------------------+--------------+--------------------+
|sudhihr|    Kasper|     Lakso|  male|1386 Verkatehtaan...|    64011|kasper.lakso@exam...|        redlion396|2005-02-17T19:55:...|    03-572-363|https://randomuse...|
|sudhihr|       Ava|    Wilson|female|1596 Moorhouse Av...|    33148|ava.wilson@exampl...|     sadmeercat469|2017-08-16T08:20:...|(212)-558-3089|https://randomuse...|
|sudhihr|  Mohammad|    Berger|  male|8456 Esplanade du...|     5391

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/crepantherx/Developer/e2e_de_kafka_to_sql/.venv/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/crepantherx/Developer/e2e_de_kafka_to_sql/.venv/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
(sel.writeStream.format("org.apache.spark.sql.cassandra")
                               .option('checkpointLocation', '/tmp/checkpoint')
                               .option('keyspace', 'spark_streams')
                               .option('table', 'created_userssss')
                               .start())